<a href="https://colab.research.google.com/github/endorgobio/IntroduccionAnaliticaPrescriptiva/blob/main/M5C12_Gurobipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="text-align: center;">
    <img alt="banner" height="230px" width="100%" src="https://github.com/endorgobio/IntroduccionAnaliticaPrescriptiva/blob/6cc6029c276aacdf228dcec4796b7b3184cfb8b7/src/header.png?raw=true" hspace="10px" vspace="0px">
</p>

# <font color='FD6E72'> **Descripción del problema** </font>





Una compañia de alimentos tiene **cinco lineas** que procesan un único tipo de producto. El plan de demanda contiene las ordenes de 10 clientes (en miles de productos) y la utilidad (en millones de pesos) que cada una de ellas genera en cada uno de las lineas


|          | Orden 1 | Orden 2 | Orden 3 | Orden 4 | Orden 5 | Orden 6 | Orden 7 | Orden 8 | Orden 9 | Orden 10 |
|----------|---------|---------|---------|---------|---------|---------|---------|---------|---------|----------|
| Cantidad |   16    |   29    |   24    |   20    |   17    |   30    |   32    |   28    |   20    |    20    |
| Utilidad 1 |   73    |   85    |   89    |   73    |   52    |   71    |   102   |   73    |   93    |    79    |
| Utilidad 2 |   67    |   79    |   88    |   65    |   50    |   72    |   98    |   70    |   90    |    75    |
| Utilidad 3 |   70    |   82    |   87    |   68    |   53    |   69    |   99    |   72    |   91    |    77    |
| Utilidad 4 |   75    |   84    |   86    |   71    |   51    |   70    |   101   |   74    |   92    |    78    |
| Utilidad 5 |   72    |   81    |   85    |   66    |   54    |   73    |   97    |   71    |   89    |    76    |


La política de la empresa estipula que a un cliente se le despacha  el total de su demanda o en caso contrarion no se le despacha nada. La capacidad de cada linea es 55 mil unidades de producto, pero alistar cualquiera de las lineas sin importar el número de productos que se produzcan en ella tiene un costo de 200. La decisión que debe tomarse es:


> **¿Cuáles de las ordenes deben producirse y en que líneas deben hacerse?**


# <font color='#FD6E72'> **Verbalización del problema** </font>



## <font color='ff6d33'>**Conjuntos** </font>
* Conjunto de pedidos.
* Conjunto de líneas de producción.


## <font color='ff6d33'>**Parámetros** </font>
- Utilidad asociada a cada pedido.
- Demanda asociada a cada pedido.
- Capacidad de cada línea de producción.
- Costo de alistamiento de la línea.

## <font color='ff6d33'>**Decisiones** </font>
  * Debe decidirse cuáles lineas activar (alistar)
  * Para cada pedido debe decidirse si producirlo o no producirlo y en caso de producirlo en cuál de las lineas debería producirse

## <font color='ff6d33'>**Objetivo** </font>

  Generar la máxima utilidad entendida como la utilidad generada por los pedidos producidos menos el costo de alistar las líneas que entran en operación

## <font color='ff6d33'>**Restricciones** </font>

  * La cantidad de unidades de producto requerida por los pedidos a producir en cada línea no debe sobrepasar su capacidad
  * Cada orden debe producirse cuando más en una linea
  * No pueden producirse pedidos en líneas que no han sido alistadas
  * Las ordenes no pueden fraccionarse

# <font color='#FD6E72'> **Formulación matemática** </font>

## <font color='ff6d33'> **Formulación generalizada** </font>

Este mismo modelo puede escribirse de manera generalizada de la siguiente frma


### **Conjuntos**
- $P$: Conjunto de pedidos.
- $L$: Conjunto de líneas de producción.

### **Parámetros**
- $d_i$: Demanda del pedido $i$, $\forall i \in P$.
- $u_{ij}$: Utilidad del pedido $i$ en la línea $j$, $\forall i \in P, \forall j \in L$.
- $c_j$: Capacidad de la línea $j$, $\forall j \in L$.
- $s$: Costo de alistamiento de la línea.

### **Variable de Decisión**


> $y_{j} =
\begin{cases}
1, & \text{si se usa (alista) la línea $j$,} \\
0, & \text{si no usa (alista) la línea $j$}
\end{cases} \quad \forall j \in L$


> $x_{ij} =  \begin{cases}
1, & \text{si el pedido $i$ se realiza en la línea $j$,} \\
0, & \text{si el pedido $i$ no se realiza en la línea $j$,}
\end{cases} \quad \forall i \in P, \forall j \in L$


### **Función Objetivo**

Maximizar la utilidad total:

> $\text{Maximizar} \quad \sum_{i \in P} \sum_{j \in L} u_{ij} x_{ij} - \sum_{j \in L} s y_j$

### **Restricciones**

- Restricción de capacidad por línea:
  > $ \sum_{i \in P} d_i x_{ij} \leq c_j, \quad \forall j \in L$

- Restricción de asignación de pedidos:
  > $\sum_{j \in L} x_{ij} \leq 1, \quad \forall i \in P$

- Restricción de alistamiento:
  > $  x_{ij} \leq y_j, \quad \forall i \in P, \forall j \in L$

- Restricción de variables binarias:
  > $x_{ij} \in \{0, 1\}, \quad \forall i \in P, \forall j \in L$





# <font color='#FD6E72'> **Instancia de datos** </font>

Consideremos una isnatncia particular para tener claridad sobre como se almacenan los datos. Posteriormente veremos que podemos automatizar la creación del modelo en una función que reciba como argumento cualquier instancia de datos

In [ ]:
# dictionary of data
data ={
  # demads
  'demands': {1: 16, 2: 29, 3: 24, 4: 20, 5: 17, 6: 30, 7: 32, 8: 28, 9: 20, 10: 20},
  # Utilities
  'utilities': {
      (1, 1): 73, (1, 2): 67, (1, 3): 70, (1, 4): 75, (1, 5): 72,
      (2, 1): 85, (2, 2): 79, (2, 3): 82, (2, 4): 84, (2, 5): 81,
      (3, 1): 89, (3, 2): 88, (3, 3): 87, (3, 4): 86, (3, 5): 85,
      (4, 1): 73, (4, 2): 65, (4, 3): 68, (4, 4): 71, (4, 5): 66,
      (5, 1): 52, (5, 2): 50, (5, 3): 53, (5, 4): 51, (5, 5): 54,
      (6, 1): 71, (6, 2): 72, (6, 3): 69, (6, 4): 70, (6, 5): 73,
      (7, 1): 102, (7, 2): 98, (7, 3): 99, (7, 4): 101, (7, 5): 97,
      (8, 1): 73, (8, 2): 70, (8, 3): 72, (8, 4): 74, (8, 5): 71,
      (9, 1): 93, (9, 2): 90, (9, 3): 91, (9, 4): 92, (9, 5): 89,
      (10, 1): 79, (10, 2): 75, (10, 3): 77, (10, 4): 78, (10, 5): 76
  },
  # Capacities of the lines
  'capacities': {1: 55, 2: 55, 3: 55, 4: 55, 5: 55},
  # Cost of setting up each line
  'cost_setup': {1: 150, 2: 150, 3: 150, 4: 150, 5: 150}
}

# <font color='#FD6E72'> **Implementación** </font>

*   El modelo se implementa en un **lenguaje de modelación**, (en este caso `pyomo`).
*   El modelose resuelve haciendo uso de un **optimizador** (en este caso `highs`).



## <font color='ff6d33'> **Instalamos las librerias necesarias** </font>

In [ ]:
!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

## <font color='ff6d33'> **Creamos el modelo** </font>

Creamos la función `create_model()` que recibe como argunmento la variable `data` que contiene la información de la instancia y retorna el modelo construido

In [ ]:
def create_model(data):
  # Create model
  model = gp.Model("Production_Optimization")

  # read ids for orders and lines
  Orders = list(data['demands'].keys())
  Lines = list(data['capacities'].keys())

  # Decision Variables
  prod = model.addVars(Orders, Lines, vtype=GRB.BINARY, name="prod")  # Order assignment variables
  setup = model.addVars(Lines, vtype=GRB.BINARY, name="setup")  # Line usage variables

  # Objective Function: Maximize total utility
  model.setObjective(
      gp.quicksum(data['utilities'][i, j] * prod[i, j] for i in Orders for j in Lines) - gp.quicksum(data['cost_setup'][j] * setup[j] for j in Lines),
      GRB.MAXIMIZE
  )

  # Constraints
  # Capacity constraint per line
  for j in Lines:
      model.addConstr(gp.quicksum(data['demands'][i] * prod[i, j] for i in Orders) <= data['capacities'][j], name=f"Capacity_{j}")

  # Each order is assigned to at most one production line
  for i in Orders:
      model.addConstr(gp.quicksum(prod[i, j] for j in Lines) <= 1, name=f"Order_Assignment_{i}")

  # Setup constraint: a line must be used if an order is assigned to it
  for i in Orders:
      for j in Lines:
          model.addConstr(prod[i, j] <= setup[j], name=f"Setup_{i}_{j}")

  return model


Creemos el modelo para la instancia de datos data:

In [ ]:
model = create_model(data)

# <font color='#FD6E72'> **Resolver el modelo** </font>

El modelo se resuelve para una instancia de datos, en este caso la instancia que hemos usado para crearlo.

In [ ]:
model.optimize()
if model.status == GRB.OPTIMAL:
    print(f"Objective value: {model.ObjVal}")
else:
    print("Optimization was not successful")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 65 rows, 55 columns and 200 nonzeros
Model fingerprint: 0x9deed8b1
Variable types: 0 continuous, 55 integer (55 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [5e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 23.0000000
Presolve removed 15 rows and 0 columns
Presolve time: 0.00s
Presolved: 50 rows, 55 columns, 190 nonzeros
Variable types: 0 continuous, 55 integer (55 binary)

Root relaxation: objective 3.871216e+02, 76 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    

# <font color='#FD6E72'> **Obtener la solución** </font>

De momento estaremos interesados en dos componentes de la solución:
* El valor de la función objetivo
* El valor de las variales de decisión

## <font color='ff6d33'>**Valor de la función objetivo** </font>

Una vez optimizado el modelo para la instancia dada, el valor de la función objetivo puede obtenerse al consultar directamente el componente asociado al objetivo, en nuestro caso este sería `model.obj()`

In [ ]:
print("Objective Function Value:", model.objVal)





Objective Function Value: 119.0


## <font color='ff6d33'>**Valor de las variables de decisión** </font>

El valor de las variables se obtiene con el metodo `.x` el cual se aplica a la variable en particular para la cual se esta interesado en obtener el valor.

Si deseamos obtener el valor de la variable que indica si el pedido 1 se realizó en la línea 4, tendriamos:


```python
prod[1,4].x
```



Sin embargo, dado que creamos el modelo a través de la función `create_model()` la variable `prod` es directamente accesisble solo en el ámbito de la función. Debemos entonces primero obtener todas las variables del modelo y allí tendriamos acceso a la variable deseada

In [ ]:
variables = model.getVars()
variables

[<gurobi.Var prod[1,1] (value 0.0)>,
 <gurobi.Var prod[1,2] (value -0.0)>,
 <gurobi.Var prod[1,3] (value 0.0)>,
 <gurobi.Var prod[1,4] (value 0.0)>,
 <gurobi.Var prod[1,5] (value 1.0)>,
 <gurobi.Var prod[2,1] (value 1.0)>,
 <gurobi.Var prod[2,2] (value -0.0)>,
 <gurobi.Var prod[2,3] (value 0.0)>,
 <gurobi.Var prod[2,4] (value 0.0)>,
 <gurobi.Var prod[2,5] (value -0.0)>,
 <gurobi.Var prod[3,1] (value 1.0)>,
 <gurobi.Var prod[3,2] (value -0.0)>,
 <gurobi.Var prod[3,3] (value 0.0)>,
 <gurobi.Var prod[3,4] (value -0.0)>,
 <gurobi.Var prod[3,5] (value -0.0)>,
 <gurobi.Var prod[4,1] (value -0.0)>,
 <gurobi.Var prod[4,2] (value -0.0)>,
 <gurobi.Var prod[4,3] (value -0.0)>,
 <gurobi.Var prod[4,4] (value -0.0)>,
 <gurobi.Var prod[4,5] (value -0.0)>,
 <gurobi.Var prod[5,1] (value 0.0)>,
 <gurobi.Var prod[5,2] (value 0.0)>,
 <gurobi.Var prod[5,3] (value 0.0)>,
 <gurobi.Var prod[5,4] (value 0.0)>,
 <gurobi.Var prod[5,5] (value 1.0)>,
 <gurobi.Var prod[6,1] (value -0.0)>,
 <gurobi.Var prod[6,2] (va

Note entonces como obtenemos el valor de las variables de decisión `prod` que indiquen la asignación de la orden a alguna línea y aquellas que indiquenlas líneas de producción que se usan

In [ ]:
for var in model.getVars():
  if var.X > 0.1:
    print(f"{var.VarName} = {var.X}")

prod[1,5] = 1.0
prod[2,1] = 1.0
prod[3,1] = 1.0
prod[5,5] = 1.0
prod[7,4] = 1.0
prod[9,4] = 1.0
prod[10,5] = 1.0
setup[1] = 1.0
setup[4] = 1.0
setup[5] = 1.0
